In [1]:
# the trained model is saved in the 'mask_rcnn_openimage_cfg_0040.h5'
# copy it to the working directory

####################################################################
#Evaluate

# mean average precision: mAP
# https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173

# IoU => Precison & Recall => PR curve => AP => mAP (mean of AP of all classes)

# I met
# Error : Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
# https://github.com/tensorflow/tensorflow/issues/24828
# Suggests the following solution
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config0 = ConfigProto()
config0.gpu_options.allow_growth = True
session = InteractiveSession(config=config0)

# evaluate the mask rcnn model on the kangaroo dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

import csv
import pandas as pd 
from PIL import Image
import numpy as np

Using TensorFlow backend.


In [2]:
image_dir = '/HDD1/Downloads/OpenImageV5/Train/'

In [3]:
data_dir = '/HDD1/Downloads/OpenImageV5/MyData/'
# read the list of image IDs
with open(data_dir+'training_image_list.csv','r') as f:
    reader = csv.reader(f)
    training_image_list = list(reader)[0]
with open(data_dir+'validation_image_list.csv','r') as f:
    reader = csv.reader(f)
    validation_image_list = list(reader)[0]
with open(data_dir+'test_image_list.csv','r') as f:
    reader = csv.reader(f)
    test_image_list = list(reader)[0]

with open(data_dir+'all_image_list.csv','r') as f:
    reader = csv.reader(f)
    all_image_list = list(reader)[0]

# read the boxes
training_dataset = pd.read_csv(data_dir+'training_dataset.csv')
validation_dataset = pd.read_csv(data_dir+'validation_dataset.csv')
test_dataset = pd.read_csv(data_dir+'test_dataset.csv')
all_dataset = pd.read_csv(data_dir+'all_dataset.csv')

In [4]:
# load the classs imformation
with open(data_dir+'class-descriptions-boxable.csv','r') as f:
    reader = csv.reader(f)
    class_file = list(reader)

In [5]:
# define a dataset class
# based on the Dataset class
class OpenImageDataset(Dataset):
    def load_dataset(self,dataset_dir,is_train=True):
        #define one class
        # add_class is built in the original class
        for i in range(len(class_file)):
            self.add_class("OpenImage",i+1,class_file[i][1])
        
        if is_train:
            # it takes too long to evaluate the entire dataset, 
            # so I just set the size of the evaluation to a small number
            for image_id in range(len(training_image_list)*0+10000):
                img_path = image_dir + all_image_list[image_id] + ".jpg"
                self.add_image('OpenImage',image_id=all_image_list[image_id],path=img_path)
        if not is_train:
            # because we do not change the hyperparameters in the project
            # I use the valiataiton and test set as test set
            for image_id in range(len(training_image_list),len(all_image_list)\
                                  *0+len(training_image_list)+10000):
                img_path = image_dir + all_image_list[image_id] + ".jpg"
                self.add_image('OpenImage',image_id=all_image_list[image_id],path=img_path)
            
    # define a function to extract the boxes from csv[ done 11.11]
    def extract_boxes(self,image_name):
        # find the lines with imageID
        image_data = all_dataset.loc[all_dataset['ImageID']==image_name]
        
        # extract image dimensions
        image=Image.open(image_dir+image_name+'.jpg')
        width,height = image.size
        
        # extract all the bounding boxes
        boxes = list()
        class_label = list()
        for index,row in image_data.iterrows():
            #because in this dataset, XMin is shown in numbers like 0.234
            xmin = round(width*row['XMin'])
            ymin = round(height*row['YMin'])
            xmax = round(width*row['XMax'])
            ymax = round(height*row['YMax'])
            boxes.append([xmin,ymin,xmax,ymax])
            class_label.append(row['LabelName'])
        
        # obtain class id 
        class_ids = list()
        for label in class_label:
            # search for the label in first column of the file
            index = np.array(class_file)[:,0].tolist().index(label)
            class_ids.append(class_file[index][1])
        
        return boxes, width, height, class_ids
    
    def load_mask(self,image_id_order):
        # from image_id to image_name
        info = self.image_info[image_id_order]
        image_name = info['id']
        # call extract_boxes tp get the bboxes and w,h
        boxes,w,h,class_ids_name = self.extract_boxes(image_name)
        # create masks
        masks = zeros([h,w,len(boxes)]) # could be multiple boxes
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            col_start, col_end = box[0],box[2]
            row_start, row_end = box[1],box[3]
            masks[row_start:row_end,col_start:col_end,i] = 1
            # record the class of each box
            class_ids.append(self.class_names.index(class_ids_name[i]))
        return masks, asarray(class_ids,dtype='int32')
    
    # image_id must be int
    def image_reference(self,image_id_order):
        info = self.image_info[image_id_order]
        return info['path']
    
    # I met a problem that the sizes of predicted masks and true masks do not match
    # finally i found it because I did not import the image and mask data from a same file
    
    # one should pay attention to this load_mask method and image reference method
    # make sure that they refer to a same image
    # better to use identical functions/structures

In [6]:
from mrcnn.config import Config
# define a configuration for the model
class PredictionConfig(Config):
    # GIve the configutation a recognizable name
    NAME = "OpenImage_cfg"
    #Number of classes 
    NUM_CLASSES = 1 + len(class_file)
    # seems that in evaluation,GPU_COUNT*IMAGES_PER_GPU has to be one?
    # bug? https://github.com/matterport/Mask_RCNN/pull/1082
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
#     Actually using this "none" is better for this dataset, because some of the images 
#     has a large width to height ratio. See the config.py  
#     IMAGE_RESIZE_MODE = "none"
    
config = PredictionConfig()

In [7]:
# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
    APs = list()
    for image_id in dataset.image_ids:
        # load image, bounding boxes and masks for the image id
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
        # convert pixel values (e.g. center)
        # why do we need this
        scaled_image = mold_image(image, cfg)
        # convert image into one sample
        sample = expand_dims(scaled_image, 0)
        # make prediction
        yhat = model.detect(sample, verbose=0)
        # extract results for first sample
        r = yhat[0]
        # calculate statistics, including AP
        AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
        
        # store
        APs.append(AP)
    # calculate the mean AP across all images
    mAP = mean(APs)
    return mAP

In [ ]:
# load the train dataset
train_set = OpenImageDataset()
train_set.load_dataset('OpenImage', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = OpenImageDataset()
test_set.load_dataset('OpenImage', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=config)
# load model weights
model.load_weights('mask_rcnn_openimage_cfg_0040.h5', by_name=True)
# evaluate model on training dataset
train_mAP = evaluate_model(train_set, model, config)
print("Train mAP: %.3f" % train_mAP)
# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model,config)
print("Test mAP: %.3f" % test_mAP)

Train: 10000
Test: 10000
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.

